# Generación de columnas

La generación de columnas no difiere mucho entre problemas. La idea básica siempre es:

(1) Tenemos un modelo que puede dividirse en un problema tipo "set cover" o "set packing" y otro que genera los sets del modelo de set cover o set packing.

Un ejemplo típico es el cutting stock. El modelo master resultante es:

$\mbox{[MIN]} \sum_{j\in \mathcal{J}} x_j$ 

$\sum_{j\in \mathcal{J}} a_{ij} x_j \geq d_i \qquad \forall i\in P$

$x_j \in \mathbb{Z}_{\geq0}\qquad \forall j\in \mathcal{J}$

donde las variables indican el número de veces que se repite el patrón en rollos diferentes y se intenta minimizart el número de rollos utilizados para cumplir con la demanda $d_i$ de los $P$ productos.

El problema de pricing usa los costes reducidos de las variables del dual para calcular el mejor rollo:

$ \mbox{[MIN]} 1 - \sum_{i \in P} \quad {\pi_i y_i}$

sujeto a:

$ \sum_{i \in P} w_i y_{i} \leq W $

$ y_i \in \mathbb{Z}_{\geq0}\qquad \forall j\in \mathcal{J} $

Que corresponde a un modelo de knapsack tradicional.

Nota: Consultar http://www4.ncsu.edu/~kksivara/ma505/handouts/gilmore-gomory1.pdf para detalles de lo que estamos haciendo.